GEO to be updated 

In [26]:
import scanpy as sc
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import figure
import numpy as np
import glob

In [27]:
sc.set_figure_params(scanpy=True, dpi=150, dpi_save=150, figsize = [7, 7], fontsize = 14, format = 'svg')

In [28]:
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

%matplotlib inline

In [29]:
samples = sorted(glob.glob('*/*/filtered/'))

In [36]:
legend = {
    'HYR__3f9a39__20210904_by_6um.Solo.out/Gene/filtered/': '6_um',
    'HYR__789b3f__20200723_by_50um.Solo.out/Gene/filtered/': '50_um',
    'HYR__c21295__20210330_mcf-7_mmel_S5.Solo.out.filtered/Gene/filtered/': '12_um'
}

In [37]:
top_n = 2500
markersize = 10

In [38]:
for sample in samples:
    mtx = sample + 'matrix.mtx'
    features = sample + 'features.tsv'
    barcodes = sample + 'barcodes.tsv'
    data = sc.read_mtx(mtx)
    data = data.T
    features = pd.read_csv(features, header=None, sep='\t')
    barcodes = pd.read_csv(barcodes, header=None)
    data.var_names = features[0]
    data.obs_names = barcodes[0]

    adata = data
    adata.var_names_make_unique()

    ## then, generate barnyard plots, but now from matrix
    # get variables list from adata
    adata_vars = list(adata.var.index.values)
    adata_obs = list(adata.obs.index.values)

    # list gene names from each genome
    adata_vars_GRCh38 = [x for x in adata_vars if x.startswith('ENSG') or x.startswith('GRCh38')]
    adata_vars_mm10 = [x for x in adata_vars if x.startswith('ENSMUS') or x.startswith('mm10')]

    # then, select from adata the values corresponding to each gene
    adata_GRCh38 = adata[:, adata_vars_GRCh38]
    adata_mm10 = adata[:, adata_vars_mm10]

    # then, colsum in order to collapse counts for each genome
    adata_GRCh38_colsum = adata_GRCh38.X.sum(axis=1)
    adata_mm10_colsum = adata_mm10.X.sum(axis=1)

    # convert to df
    df_adata_GRCh38_colsum = pd.DataFrame(adata_GRCh38_colsum, index = adata_obs, columns = ['GRCh38'])
    df_adata_mm10_colsum = pd.DataFrame(adata_mm10_colsum, index = adata_obs, columns = ['mm10'])

    # merge
    df_adata_colsum = pd.concat([df_adata_GRCh38_colsum, df_adata_mm10_colsum], axis=1, sort=False)

    # add rowsum
    df_adata_colsum['sum'] = df_adata_colsum.sum(axis=1)

    # now sort on rowsum
    df_adata_colsum_sorted = df_adata_colsum.sort_values(by = 'sum', axis = 0, ascending = False)

    sns.set_style('darkgrid')
    sns.set_context('talk')
    ddmarkersize=25
    nbins = 10
    # now replace zeros by 1s for the log scale
    df_adata_colsum_nozeros = df_adata_colsum_sorted.mask(df_adata_colsum_sorted == 0, 1)
    df_adata_colsum_nozeros_sorted = df_adata_colsum_nozeros.sort_values(by = 'sum', axis = 0, ascending = False)

    ## now plot the barnyard for the top n cells 
    df_adata_colsum_nozeros_sorted_top_n = df_adata_colsum_nozeros_sorted

    x = df_adata_colsum_nozeros_sorted_top_n['GRCh38']
    y = df_adata_colsum_nozeros_sorted_top_n['mm10']

    xy = np.vstack([np.log10(x),y])
    z = gaussian_kde(xy)(xy)
    bins = np.linspace(z.min(), z.max(), nbins)
    digitized = np.digitize(z, bins)
    bin_means = [z[digitized == i].mean() for i in range(1, len(bins))]
    digitized_rescaled = np.interp(digitized, (digitized.min(), digitized.max()), (z.min(), z.max()))

    ax = sns.scatterplot(x = 'GRCh38', y = 'mm10', data = df_adata_colsum_nozeros_sorted_top_n, hue = digitized_rescaled, palette='viridis', edgecolor=None, s=markersize)

    # sns.kdeplot(x = x, y = y, shade=False, log_scale=True, levels=5, cbar=False, bw_method=0.1, ax=axes[1])
    ax.set_title(legend[sample])
    scalemax = max([max(x), max(y)])
    scalemin = -0.05*max([max(x), max(y)])
    ax.set_xlim([scalemin, scalemax])
    ax.set_ylim([scalemin, scalemax])
    ax.set(xscale='linear')
    ax.set(yscale='linear')

    ax.set_aspect('equal', adjustable='box')
    ax.set_aspect(1./ax.get_data_ratio())

    ax.set_xlabel('hg38 UMIs')
    ax.set_ylabel('mm10 UMIs')

    plt.savefig('plts_pub/barnyard_'+legend[sample]+'_'+str(markersize)+'.svg', dpi=150)
    plt.show()
    plt.close()
    
    
    df_adata_colsum_nozeros_sorted_top_n['purity'] = df_adata_colsum_nozeros_sorted_top_n['GRCh38']/df_adata_colsum_nozeros_sorted_top_n['sum']
    cells_pure = len(df_adata_colsum_nozeros_sorted_top_n[df_adata_colsum_nozeros_sorted_top_n['purity'] >= 0.95]) + len(df_adata_colsum_nozeros_sorted_top_n[df_adata_colsum_nozeros_sorted_top_n['purity'] <=0.05])
    print(sample)
    print(cells_pure/len(df_adata_colsum_nozeros_sorted_top_n))
    print(df_adata_colsum_nozeros_sorted_top_n.median())

FileNotFoundError: [Errno 2] No such file or directory: 'HYR__3f9a39__20210904_by_6um.Solo.out/Gene/filtered/matrix.mtx'

In [35]:
for sample in samples:
    mtx = sample + 'matrix.mtx'
    features = sample + 'features.tsv'
    barcodes = sample + 'barcodes.tsv'
    data = sc.read_mtx(mtx)
    data = data.T
    features = pd.read_csv(features, header=None, sep='\t')
    barcodes = pd.read_csv(barcodes, header=None)
    data.var_names = features[0]
    data.obs_names = barcodes[0]

    adata = data
    adata.var_names_make_unique()

    ## then, generate barnyard plots, but now from matrix
    # get variables list from adata
    adata_vars = list(adata.var.index.values)
    adata_obs = list(adata.obs.index.values)

    # list gene names from each genome
    adata_vars_GRCh38 = [x for x in adata_vars if x.startswith('ENSG') or x.startswith('GRCh38')]
    adata_vars_mm10 = [x for x in adata_vars if x.startswith('ENSMUS') or x.startswith('mm10')]

    # then, select from adata the values corresponding to each gene
    adata_GRCh38 = adata[:, adata_vars_GRCh38]
    adata_mm10 = adata[:, adata_vars_mm10]

    # then, colsum in order to collapse counts for each genome
    adata_GRCh38_colsum = adata_GRCh38.X.sum(axis=1)
    adata_mm10_colsum = adata_mm10.X.sum(axis=1)

    # convert to df
    df_adata_GRCh38_colsum = pd.DataFrame(adata_GRCh38_colsum, index = adata_obs, columns = ['GRCh38'])
    df_adata_mm10_colsum = pd.DataFrame(adata_mm10_colsum, index = adata_obs, columns = ['mm10'])

    # merge
    df_adata_colsum = pd.concat([df_adata_GRCh38_colsum, df_adata_mm10_colsum], axis=1, sort=False)

    # add rowsum
    df_adata_colsum['sum'] = df_adata_colsum.sum(axis=1)

    # now sort on rowsum
    df_adata_colsum_sorted = df_adata_colsum.sort_values(by = 'sum', axis = 0, ascending = False)

    sns.set_style('darkgrid')
    sns.set_context('talk')
    ddmarkersize=25
    nbins = 10
    # now replace zeros by 1s for the log scale
    df_adata_colsum_nozeros = df_adata_colsum_sorted.mask(df_adata_colsum_sorted == 0, 1)
    df_adata_colsum_nozeros_sorted = df_adata_colsum_nozeros.sort_values(by = 'sum', axis = 0, ascending = False)

    ## now plot the barnyard for the top n cells 
    df_adata_colsum_nozeros_sorted_top_n = df_adata_colsum_nozeros_sorted

    pd.to_csv(legend[sample] + '.tsv', df_adata_colsum_nozeros_sorted_top_n, sep='\t', header=False, index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'HYR__3f9a39__20210904_by_6um.Solo.out/Gene/filtered/matrix.mtx'